In [1]:
import pandas as pd
import sys
import psycopg2
import numpy as np

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [3]:
# Connection parameters, yours will be different
param_dic = {
    "host"      : "stockschapnes.postgres.database.azure.com",
    "database"  : "postgres",
    "user"      : "chapnes",
    "password"  : "House333!",
    "port" : "5432"
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    print("Connection successful")
    return conn




def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1

    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()

    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df



# Connect to the database
conn = connect(param_dic)
# column_names = ["id", "source", "datetime", "mean_temp"]
column_names = ["id", "p18_05_20_01", "p19_05_20_06", "p19_10_21_01", "p20_05_21_10", "p21_06_22_01", "p22_01", "option"]

# Execute the "SELECT *" query
# df = postgresql_to_dataframe(conn, "select * from test_100_beta_more_09", column_names)



# test_100 = postgresql_to_dataframe(conn, "select * from test_100_common_cl_5", column_names)
# test_100 = postgresql_to_dataframe(conn, "select * from test_100_common_cl_2", column_names)
# test_100 = postgresql_to_dataframe(conn, "select * from cr_test_100_common_cl_2", column_names)
# test_100 = postgresql_to_dataframe(conn, "select * from cr_test_100_new_cl_2", column_names)
# test_100 = postgresql_to_dataframe(conn, "select * from cr_test_100_common_cl_4", column_names)
# test_100 = postgresql_to_dataframe(conn, "select * from cr_test_100_common_cl_3", column_names)
test_100 = postgresql_to_dataframe(conn, "select * from test_day_100_common_cl_4", column_names)


# df

Connecting to the PostgreSQL database...
Connection successful


In [4]:
test_100.head(3)

,id,p18_05_20_01,p19_05_20_06,p19_10_21_01,p20_05_21_10,p21_06_22_01,p22_01,option
0,1,13.883370680683003,9.716101420387,68.26629246201362,54.46348701819803,-14.600031988391207,-10.000280778660972,"gf_copy.iloc[ind[i], 4] > gf_copy.iloc[ind[i] ..."
1,2,2.7467012350404447,-6.1894353752980455,11.512833676127316,18.302276275246836,3.9823315082047697,-11.551606814445009,"gf_copy.iloc[ind[i], 4] > gf_copy.iloc[ind[i] ..."
2,3,12.94909161864859,1.7243540167811355,14.810890471271284,23.592668492966066,7.554279872254024,-4.756054241349841,"gf_copy.iloc[ind[i], 4] > gf_copy.iloc[ind[i] ..."


In [5]:
def get_plus_results(df):
    df_more_0_pre = df[(df.p18_05_20_01 > 0)
                         & (df.p19_05_20_06 > 0)
                         & (df.p19_10_21_01 > 0)
                         & (df.p20_05_21_10 > 0)
                         & (df.p21_06_22_01 > 0)
                         & (df.p22_01 > 0)]


    df_more_0 = df_more_0_pre.copy()
    
    df_more_0['p18_05_20_01'] = df_more_0['p18_05_20_01'].astype(float)
    df_more_0['p19_05_20_06'] = df_more_0['p19_05_20_06'].astype(float)
    df_more_0['p19_10_21_01'] = df_more_0['p19_10_21_01'].astype(float)
    df_more_0['p20_05_21_10'] = df_more_0['p20_05_21_10'].astype(float)
    df_more_0['p21_06_22_01'] = df_more_0['p21_06_22_01'].astype(float)
    df_more_0['p22_01'] = df_more_0['p22_01'].astype(float)    
    
    df_more_0['Sum'] = df_more_0['p18_05_20_01'] + df_more_0['p19_05_20_06'] + df_more_0['p19_10_21_01'] + df_more_0['p20_05_21_10'] + df_more_0['p21_06_22_01'] + df_more_0['p22_01'] 
    df_more_0 = df_more_0.sort_values(by=['Sum'], ascending=False)
    return df_more_0
    
    

more_0 = get_plus_results(test_100)


In [6]:
def get_plus_options(df):
    df_more_150 = df[(df.Sum > 50)]
    return df_more_150['option'].values


plus = get_plus_options(more_0)



In [7]:
plus

array(['gf_copy.iloc[ind[i], 4] > (gf_copy.iloc[ind[i] - 1, 4] + 3)',
       'gf_copy.iloc[ind[i], 4] > (gf_copy.iloc[ind[i] - 1, 4] + 1)',
       'gf_copy.iloc[ind[i], 4] > (gf_copy.iloc[ind[i] - 1, 4] + 1)  > (gf_copy.iloc[ind[i] - 2, 4] - 1)',
       'gf_copy.iloc[ind[i], 4] > (gf_copy.iloc[ind[i] - 1, 4] + 1.5) and gf_copy.iloc[ind[i] - 1, 4] > (gf_copy.iloc[ind[i] - 2, 4] - 2)',
       'gf_copy.iloc[ind[i], 4] > (gf_copy.iloc[ind[i] - 1, 4] + 1.5) and gf_copy.iloc[ind[i] - 1, 4] > (gf_copy.iloc[ind[i] - 2, 4] - 1.5)'],
      dtype=object)

In [8]:
more_0

,id,p18_05_20_01,p19_05_20_06,p19_10_21_01,p20_05_21_10,p21_06_22_01,p22_01,option,Sum
9,10,0.039371,49.893315,69.231603,25.272671,0.134057,18.992392,"gf_copy.iloc[ind[i], 4] > (gf_copy.iloc[ind[i]...",163.563409
5,6,6.478673,32.603955,56.093278,31.112447,2.146903,10.745133,"gf_copy.iloc[ind[i], 4] > (gf_copy.iloc[ind[i]...",139.180389
25,26,2.868761,1.118211,23.193176,27.076674,4.757566,7.138146,"gf_copy.iloc[ind[i], 4] > (gf_copy.iloc[ind[i]...",66.152534
34,35,1.323218,12.333924,21.568512,21.033864,1.466699,8.330800,"gf_copy.iloc[ind[i], 4] > (gf_copy.iloc[ind[i]...",66.057017
31,32,0.785478,12.333918,21.676684,21.142058,1.454378,7.387814,"gf_copy.iloc[ind[i], 4] > (gf_copy.iloc[ind[i]...",64.780331


In [9]:
# more_0.iloc[0,7]

In [10]:
def get_minus_results(df):


    df_less_0_pre = df[(df.p18_05_20_01 < 5)
                         & (df.p19_05_20_06 < 5)
                         & (df.p19_10_21_01 < 5)
                         & (df.p20_05_21_10 < 5)
                         & (df.p21_06_22_01 < 5)
                         & (df.p22_01 < 5)]

    df_less_0 = df_less_0_pre.copy()
    
    df_less_0['Sum'] = df_less_0['p18_05_20_01'] + df_less_0['p19_05_20_06'] + df_less_0['p19_10_21_01'] + df_less_0['p20_05_21_10'] + df_less_0['p21_06_22_01'] + df_less_0['p22_01'] 
    df_less_0 = df_less_0.sort_values(by=['Sum'], ascending=True)
    
    return df_less_0




less_5 = get_minus_results(test_100)


In [11]:
def get_minus_options(df):
    df_less_150 = df[(df.Sum < -16)]
    return df_less_150['option'].values



minus = get_minus_options(less_5)


In [12]:
minus

array([], dtype=object)

In [13]:
less_5

,id,p18_05_20_01,p19_05_20_06,p19_10_21_01,p20_05_21_10,p21_06_22_01,p22_01,option,Sum
